In [1]:
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df=pd.read_csv('../balanced_cleaned_trian_dataset.csv',index_col=0)
test_set=pd.read_csv('../testset.csv',index_col=0)

In [3]:
grouped = df.groupby("smoking")
df_0 = grouped.get_group(0)
df_1 = grouped.get_group(1)
df_0_sub = df_0.sample(n=150, random_state=42)
df_1_sub = df_1.sample(n=150, random_state=42)
df_sub = pd.concat([df_0_sub, df_1_sub])
df_sub = df_sub.sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
train_copy=df_sub.drop('id',axis=1)
X = train_copy.drop('smoking', axis=1)
y = train_copy['smoking']

test_set=test_set.drop('id',axis=1)
X_test= test_set.drop('smoking', axis=1)
y_test = test_set['smoking']

In [5]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 50, 300, step=10)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3, log=True)

    scores = []
    F1=[]

    for train_idx, val_idx in skf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        
        # scaler = MinMaxScaler()
        # X_train_scaled = scaler.fit_transform(X_train)
        # X_val_scaled = scaler.transform(X_val)

    
        model = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_jobs=-1,
            random_state=42
        )

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        acc = accuracy_score(y_val, y_pred)
        
        F1c=f1_score(y_val, y_pred, average='macro') 
        F1.append(F1c)
        

        y_pred_train = model.predict(X_train)
        acc_train = accuracy_score(y_train, y_pred_train)
        f1_train = f1_score(y_train, y_pred_train, average='macro')
        

        overfit_penalty = abs(acc_train - acc)
        score = 0.5 * acc - 0.5 * overfit_penalty
        
        
        
        scores.append(score)

    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize',sampler=sampler)

study.optimize(objective, n_trials=20)  


print("best_params:", study.best_params)
print(f"best_value: {study.best_value:.4f}")

[I 2025-08-22 15:59:12,935] A new study created in memory with name: no-name-8ef808dd-3657-4739-9ba5-48fb3f697573
[I 2025-08-22 15:59:13,328] Trial 0 finished with value: 0.23666666666666666 and parameters: {'n_estimators': 140, 'max_depth': 10, 'learning_rate': 0.1001303991139125}. Best is trial 0 with value: 0.23666666666666666.
[I 2025-08-22 15:59:13,611] Trial 1 finished with value: 0.2666666666666667 and parameters: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.009470040922904443}. Best is trial 1 with value: 0.2666666666666667.
[I 2025-08-22 15:59:13,819] Trial 2 finished with value: 0.22666666666666666 and parameters: {'n_estimators': 60, 'max_depth': 9, 'learning_rate': 0.05859268690985102}. Best is trial 1 with value: 0.2666666666666667.
[I 2025-08-22 15:59:14,039] Trial 3 finished with value: 0.21000000000000002 and parameters: {'n_estimators': 230, 'max_depth': 2, 'learning_rate': 0.2652261985899885}. Best is trial 1 with value: 0.2666666666666667.
[I 2025-08-22 1

best_params: {'n_estimators': 300, 'max_depth': 2, 'learning_rate': 0.005180951212267571}
best_value: 0.3133


In [6]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for train_idx, val_idx in skf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=2,
        learning_rate=0.01608746833581257,
        n_estimators=100,
        eval_metric='logloss',
        random_state=42,
        min_child_weight=5,         
        subsample=0.8,             
        colsample_bytree=0.7,       
        
        n_jobs=-1
    )
    model.fit(X_train, y_train)  
    y_test_pred = model.predict(X_val)

    acc_test = accuracy_score(y_val, y_test_pred)
    precision_test = precision_score(y_val, y_test_pred, average='macro')
    recall_test = recall_score(y_val, y_test_pred, average='macro')
    f1_test = f1_score(y_val, y_test_pred, average='macro')

    train_pred = model.predict(X_train)  
    train_acc = accuracy_score(y_train, train_pred)
    print(f"Train Accuracy: {train_acc:.4f}, test Accuracy: {acc_test:.4f}")

    print(f"test Accuracy:  {acc_test:.4f}")
    print(f"test Precision: {precision_test:.4f}")
    print(f"test Recall:    {recall_test:.4f}")
    print(f"test F1 Score:  {f1_test:.4f}")

Train Accuracy: 0.7917, test Accuracy: 0.6333
test Accuracy:  0.6333
test Precision: 0.6389
test Recall:    0.6333
test F1 Score:  0.6296
Train Accuracy: 0.8083, test Accuracy: 0.7500
test Accuracy:  0.7500
test Precision: 0.7747
test Recall:    0.7500
test F1 Score:  0.7442
Train Accuracy: 0.8042, test Accuracy: 0.6667
test Accuracy:  0.6667
test Precision: 0.7131
test Recall:    0.6667
test F1 Score:  0.6475
Train Accuracy: 0.7750, test Accuracy: 0.8000
test Accuracy:  0.8000
test Precision: 0.8013
test Recall:    0.8000
test F1 Score:  0.7998
Train Accuracy: 0.7958, test Accuracy: 0.8000
test Accuracy:  0.8000
test Precision: 0.8013
test Recall:    0.8000
test F1 Score:  0.7998


In [7]:
model = xgb.XGBClassifier(
        objective='binary:logistic',
        max_depth=2,
        learning_rate=0.01608746833581257,
        n_estimators=100,
        eval_metric='logloss',
        random_state=42,
        min_child_weight=5,         
        subsample=0.8,             
        colsample_bytree=0.7,       
        
        n_jobs=-1
    )
model.fit(X, y)  

y_test_pred = model.predict(X_test)

acc_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred, average='macro')
recall_test = recall_score(y_test, y_test_pred, average='macro')
f1_test = f1_score(y_test, y_test_pred, average='macro')

train_pred = model.predict(X)  
train_acc = accuracy_score(y, train_pred)
print(f"Train Accuracy: {train_acc:.4f}, test Accuracy: {acc_test:.4f}")

print(f"test Accuracy:  {acc_test:.4f}")
print(f"test Precision: {precision_test:.4f}")
print(f"test Recall:    {recall_test:.4f}")
print(f"test F1 Score:  {f1_test:.4f}")
print("\n XGBoost test classification report:")
print(classification_report(y_test, y_test_pred, digits=4))

Train Accuracy: 0.7800, test Accuracy: 0.7540
test Accuracy:  0.7540
test Precision: 0.7603
test Recall:    0.7540
test F1 Score:  0.7525

 XGBoost test classification report:
              precision    recall  f1-score   support

           0     0.8009    0.6760    0.7332      1000
           1     0.7197    0.8320    0.7718      1000

    accuracy                         0.7540      2000
   macro avg     0.7603    0.7540    0.7525      2000
weighted avg     0.7603    0.7540    0.7525      2000

